In [19]:
import numpy as np
import tensorflow as tf

# Generate a dataset of Tic-Tac-Toe boards and labels
num_examples = 1000
X = np.zeros((num_examples, 9))
y = np.zeros(num_examples)
for i in range(num_examples):
  # Generate a random Tic-Tac-Toe board
  board = np.random.randint(0, 2, size=(3, 3))
  X[i] = board.flatten()
  # Calculate the label for the board
  if np.any(np.sum(board, axis=0) == 3) or np.any(np.sum(board, axis=1) == 3) or np.sum(np.diag(board)) == 3 or np.sum(np.diag(np.fliplr(board))) == 3:
    y[i] = 1  # player 1 wins
  elif np.any(np.sum(board, axis=0) == -3) or np.any(np.sum(board, axis=1) == -3) or np.sum(np.diag(board)) == -3 or np.sum(np.diag(np.fliplr(board))) == -3:
    y[i] = 2  # player 2 wins
  else:
    y[i] = 0  # draw

# Shuffle the dataset
indices = np.arange(num_examples)
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

# Create the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, input_shape=(9,), activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10)

def print_board(board):
  """Print the Tic-Tac-Toe board"""
  print("  0 1 2")
  for i in range(3):
    print(i, end=" ")
    row = " ".join(["X" if cell == 1 else "O" if cell == 2 else " " for cell in board[i]])
    print(row)

def play_game(model):
  # Initialize the Tic-Tac-Toe board
  board = np.zeros((3, 3))

  # Loop until the game is over
  game_over = False
  while not game_over:
    # Player 1's turn
    print("Player 1's turn")
    print_board(board)  # Print the current board state
    row, col = get_move(board)
    board[row, col] = 1

    # Check if the game is over
    if check_win(board, 1):
      print("Player 1 wins!")
      game_over = True
      break
    if check_draw(board):
      print("It's a draw.")
      game_over = True
      break

    # Player 2's turn (AI)
    print("Player 2's turn (AI)")
    input = board.flatten()
    prediction = model.predict(input.reshape(1, -1))
    action = np.argmax(prediction)
    row, col = action // 3, action % 3
    # Skip the AI's turn if the move is not valid
    if board[row, col] != 0:
      continue
    board[row, col] = 2

    # Check if the game is over
    if check_win(board, 2):
      print("Player 2 wins!")
      game_over = True
      break
    if check_draw(board):
      print("It's a draw.")
      game_over = True
      break


def check_win(board, player):
  """Check if player has won"""
  # Check rows
  for i in range(3):
    if np.all(board[i] == player):
      return True
  # Check columns
  for j in range(3):
    if np.all(board[:, j] == player):
      return True
  # Check diagonals
  if np.all(np.diag(board) == player) or np.all(np.diag(np.fliplr(board)) == player):
    return True
  return False

def check_draw(board):
  """Check if the game is a draw"""
  return not np.any(board == 0)

def get_move(board):
  """Prompt the player to enter their next move"""
  while True:
    try:
      row = int(input("Enter the row (0-2): "))
      col = int(input("Enter the column (0-2): "))
      if row in range(3) and col in range(3) and board[row, col] == 0:
        return row, col
      print("Invalid move. Try again.")
    except ValueError:
      print("Invalid input. Try again.")

# Play the game
play_game(model)

Epoch 1/10
32/32 [==============================] - 0s 2ms/step - loss: 1.0016 - accuracy: 0.4330
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 0.9001 - accuracy: 0.4670
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.8296 - accuracy: 0.4640
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.7708 - accuracy: 0.4720
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7161 - accuracy: 0.6130
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6746 - accuracy: 0.6000
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6436 - accuracy: 0.6340
Epoch 8/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6150 - accuracy: 0.6720
Epoch 9/10
32/32 [==============================] - 0s 2ms/step - loss: 0.5822 - accuracy: 0.7290
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 0.5458 - accuracy: 0.7470
Player 1's turn
  0